In [1]:
import os
from pymongo import MongoClient
from dotenv import load_dotenv
import pandas as pd

In [2]:
# Muat file .env untuk mendapatkan URL MongoDB
load_dotenv()

mongodb_url = os.getenv('URL_LOCAL')
db_name = 'youtube_rewind_indonesia'
collection_name = 'full_labeling_all_new'

# Koneksi ke MongoDB
client = MongoClient(mongodb_url)
db = client[db_name]
collection = db[collection_name]

In [3]:
# Baca file CSV yang berisi data normalisasi
df_clean = pd.read_csv('../../dictionary/normalisasi-full.csv')

# Buat kamus untuk normalisasi
normalization_dict = pd.Series(df_clean['normalisasi'].values, index=df_clean['word']).to_dict()

# Fungsi untuk normalisasi teks
def normalize_text(text, normalization_dict):
    words = text.split()
    normalized_words = [normalization_dict.get(word, word) for word in words]
    return ' '.join(normalized_words)

# Cari dokumen dengan textOriginal tidak null
documents = collection.find({"textOriginal": {"$ne": None}})

# Lakukan normalisasi dan perbarui dokumen
for doc in documents:
    original_text = doc['textOriginal']
    normalized_text = normalize_text(original_text, normalization_dict)
    
    # Perbarui dokumen di MongoDB
    collection.update_one({"_id": doc["_id"]}, {"$set": {"textOriginal": normalized_text}})

print("Normalisasi selesai.")


Normalisasi selesai.
